In [9]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn

# === Paths ===
CHECKPOINT = "checkpoints/generator_epoch40.pth"  # your saved .pth
INPUT_DIR   = "processed_dataset_1/test_1/sketch"                      # folder of input sketches
OUTPUT_DIR  = "output_images"                    # where to save results
os.makedirs(OUTPUT_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)
# === Device ===
from torchvision import models

class ResNetUNetGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        base_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        # Encoder
        self.input_block = nn.Sequential(base_model.conv1, base_model.bn1, base_model.relu)
        self.enc1 = base_model.layer1
        self.enc2 = base_model.layer2
        self.enc3 = base_model.layer3
        self.enc4 = base_model.layer4
        # Decoder
        def up_block(in_ch, out_ch):
            return nn.Sequential(
                nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True)
            )
        self.up1 = up_block(512, 256)
        self.up2 = up_block(256 + 256, 128)
        self.up3 = up_block(128 + 128, 64)
        self.up4 = up_block(64 + 64, 64)
        self.final = nn.Sequential(nn.Conv2d(64, 3, kernel_size=1), nn.Tanh())

    def forward(self, x):
        x1 = self.input_block(x)  # 64x128x128
        x2 = self.enc1(x1)        # 64x128x128
        x3 = self.enc2(x2)        # 128x64x64
        x4 = self.enc3(x3)        # 256x32x32
        x5 = self.enc4(x4)        # 512x16x16
        u1 = self.up1(x5)         # 256x32x32
        u2 = self.up2(torch.cat([u1, x4], dim=1))  # 128x64x64
        u3 = self.up3(torch.cat([u2, x3], dim=1))  # 64x128x128
        u4 = self.up4(torch.cat([u3, x2], dim=1))  # 64x256x256
        return self.final(u4) #  3×256×256

# ─── Load Weights ────────────────────────────────────────────────────────────────
    gen = ResNetUNetGenerator().to(DEVICE)
    sd  = torch.load(CHECKPOINT, map_location=DEVICE)
    gen.load_state_dict(sd)
    gen.eval()


# ─── Inference Loop ─────────────────────────────────────────────────────────────
for fname in os.listdir(INPUT_DIR):
    if not fname.lower().endswith(".png"):
        continue
    path_in  = os.path.join(INPUT_DIR, fname)
    x = preprocess_sketch(path_in, size=256)

    with torch.no_grad():
        out = gen(x)  # (1,3,256,256), in [-1,1]
    # denormalize → [0,255]
    out = (out * 0.5 + 0.5).clamp(0,1) * 255
    out_np = out[0].permute(1,2,0).cpu().numpy().astype(np.uint8)

    save_fp = os.path.join(OUTPUT_DIR, fname.replace(".png", ".jpg"))
    cv2.imwrite(save_fp, cv2.cvtColor(out_np, cv2.COLOR_RGB2BGR))
    print("Saved:", save_fp)


Using device: cpu


C:\Users\Suyash Tambe\AppData\Local\Temp\ipykernel_3724\899966524.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd  = torch.load(CHECKPOINT, map_location=DEVICE)


Saved: output_images\10.jpg
Saved: output_images\175.jpg
Saved: output_images\2.jpg
Saved: output_images\222.jpg
Saved: output_images\3.jpg
Saved: output_images\70.jpg
Saved: output_images\80.jpg
Saved: output_images\9.jpg


In [10]:
import torch
import torchvision

# Check if CUDA is available (for GPU users)
print(f"CUDA Available: {torch.cuda.is_available()}")

# Print the versions to confirm correct installation
print(f"PyTorch Version: {torch.__version__}")
print(f"torchvision Version: {torchvision.__version__}")

# Test a simple operation to ensure everything is working
x = torch.rand(3, 3)
print("Random Tensor:\n", x)


CUDA Available: False
PyTorch Version: 2.5.0+cpu
torchvision Version: 0.20.0+cpu
Random Tensor:
 tensor([[0.6245, 0.7438, 0.9863],
        [0.5914, 0.3860, 0.2183],
        [0.7413, 0.9910, 0.8111]])
